### Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import opti_recruit.feature_engineering as fe
import opti_recruit.get_team_features as gtf
import opti_recruit.similarity as si
import pandas as pd
import pickle 
from sklearn.pipeline import Pipeline,make_pipeline,make_union
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from opti_recruit.data import get_data, clean_data
import numpy as np

### Loading Data

In [3]:
dfs = get_data()
dfs.keys()

dict_keys([17, 18, 19, 20, 21, 22])

In [4]:
df22 = dfs[22]
df21 = dfs[21]

In [5]:
df22_new = fe.add_features(df22)
df21_new = fe.add_features(df21)

In [6]:
df22_new.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,defending_standing_tackle,defending_sliding_tackle,is_bench,potential_diff,att_rate,def_rate,age_bin,prefered_pos,player_pos,new_nationality
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,...,35,24,False,0,Medium,Low,30-34,RW,ATT,Argentina
1,188545,R. Lewandowski,ST,92,92,119500000.0,270000.0,32,185,81,...,42,19,False,0,High,Medium,30-34,ST,ATT,Poland
2,20801,Cristiano Ronaldo,"ST, LW",91,91,45000000.0,270000.0,36,187,83,...,32,24,False,0,High,Low,Over 34,ST,ATT,Portugal
3,190871,Neymar Jr,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,...,32,29,False,0,High,Medium,25-29,LW,ATT,Brazil
4,192985,K. De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,...,65,53,False,0,High,High,30-34,CM,MID,Others


### Data Preparation

In [ ]:
for index, col in enumerate(df22_new.columns):
    print(f"{col:<26} | ", end="")
    if (index % 3 == 2):
        print("\n", end="")

In [7]:
to_drop = ['sofifa_id','short_name','player_positions','height_cm','weight_kg','club_team_id'
          ,'club_name' ,'league_name','club_position','club_joined','club_contract_valid_until','nationality_name'
          ,'nation_team_id','preferred_foot','weak_foot','work_rate','body_type','player_tags','player_traits'
          ,'is_bench','potential_diff','age_bin','player_pos','new_nationality'
          ,'value_eur','wage_eur','release_clause_eur']
#             'age']

In [8]:
df22_num = df22_new.drop(to_drop, axis = 1)

In [9]:
df22_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   overall                      19239 non-null  int64   
 1   potential                    19239 non-null  int64   
 2   age                          19239 non-null  int64   
 3   league_level                 19178 non-null  float64 
 4   skill_moves                  19239 non-null  int64   
 5   international_reputation     19239 non-null  int64   
 6   pace                         17107 non-null  float64 
 7   shooting                     17107 non-null  float64 
 8   passing                      17107 non-null  float64 
 9   dribbling                    17107 non-null  float64 
 10  defending                    17107 non-null  float64 
 11  physic                       17107 non-null  float64 
 12  attacking_crossing           19239 non-null  int64   
 13  a

### Pipeline for Numerical DFrame

In [10]:
from sklearn.pipeline import Pipeline,make_pipeline,make_union
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import set_config; set_config(display='diagram')



num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
num_col = make_column_selector(dtype_include=['float64','int64'])

cat_transformer = OneHotEncoder()
cat_col = make_column_selector(dtype_include=['object','category'])

preproc_basic = make_column_transformer(
    (num_transformer, num_col),
    (cat_transformer, cat_col),
    remainder='passthrough')


preproc_full = make_union(preproc_basic)
# preproc_full


In [11]:
SimpleImputer.get_feature_names_out = (lambda self, names=None: self.feature_names_in_)

In [12]:
X_train_transformed = preproc_basic.fit_transform(df22_num)


test_df = pd.DataFrame(X_train_transformed, 
             columns=preproc_basic.get_feature_names_out()
            )


In [13]:
def get_index(x):
    return df22[df22['sofifa_id']==x].index.tolist()[0]

In [14]:
get_index(158023)

0

In [15]:
test_df.head()

,pipeline__overall,pipeline__potential,pipeline__age,pipeline__league_level,pipeline__skill_moves,pipeline__international_reputation,pipeline__pace,pipeline__shooting,pipeline__passing,pipeline__dribbling,...,onehotencoder__prefered_pos_GK,onehotencoder__prefered_pos_LB,onehotencoder__prefered_pos_LM,onehotencoder__prefered_pos_LW,onehotencoder__prefered_pos_LWB,onehotencoder__prefered_pos_RB,onehotencoder__prefered_pos_RM,onehotencoder__prefered_pos_RW,onehotencoder__prefered_pos_RWB,onehotencoder__prefered_pos_ST
0,3.957501,3.60178,1.851089,-0.4746,2.146241,10.525295,1.628331,2.992852,3.548135,3.564477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3.812154,3.43747,1.429869,-0.4746,2.146241,10.525295,0.949332,2.992852,2.284233,2.575529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3.666806,3.27316,2.272309,-0.4746,3.448937,10.525295,1.822331,3.143798,2.389558,2.795295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3.666806,3.27316,0.798039,-0.4746,3.448937,10.525295,2.210331,2.313597,3.021509,3.454594,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.666806,3.27316,1.008649,-0.4746,2.146241,7.830520,0.755332,2.540015,3.758786,2.795295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine Similarity

#### Cosine Similarity Matrix 

In [16]:
from sklearn.metrics.pairwise import cosine_similarity 
similarities = cosine_similarity(test_df)

#### Cricri 

In [ ]:
playerIndex = similarities[2,:].argsort()[-16:][::-1][1:]
for i in playerIndex:
        print(df22_new.loc[i,'short_name'])

In [ ]:
index = get_index('L. Messi')
playerIndex = similarities[index,:].argsort()[-11:][::-1][1:]
for i in playerIndex:
        print(df22_new.loc[i,'short_name'])

In [17]:
def normalize(array):
    return np.array([round(num, 2) for num in (array - min(array))*100/(max(array)-min(array))])

In [ ]:
norm_simili= normalize(similarities[index,:])

In [ ]:
np.sort(similarities[index,:])[-11:][::-1][1:]

In [ ]:
np.sort(norm_simili)[-11:][::-1][1:]

In [ ]:
dict(zip(similarities[index,:].argsort()[-11:][::-1][1:],
    np.sort(norm_simili)[-11:][::-1][1:]))

In [ ]:
# Do not keep the first value as its the player similitude
similarities[index,:].argsort()[-11:][::-1][1:]

In [ ]:
def cosine_recommendation(player):
    index = get_index(player)
    norm_sim_array= np.sort(normalize(similarities[index,:]))[-11:][::-1][1:]
    index_matrix = similarities[index,:].argsort()[-11:][::-1][1:]
    return {'index': index_matrix,'score':norm_sim_array}

In [ ]:
df22_new.iloc[3]['short_name']

In [ ]:
pd.DataFrame(cosine_recommendation('K. Havertz'))

In [ ]:
# input_df = get_data()[22]
# df = fe.add_features(input_df)
# similarities = si.get_similarity_dataframe(df)

In [ ]:
(similarities)

In [ ]:

with open("../similarity_matrix.pickle", 'rb') as file:
    simili = pickle.load(file)

In [ ]:
len(simili)

In [34]:
res = si.cosine_recommendation('K. Havertz',similarities,df22)

In [ ]:
res.reset_index().to_dict()

In [ ]:
! pip install ipywidgets

In [18]:
from tqdm import tqdm_notebook as tqdm
import ipywidgets 

In [26]:
# create a dictionary per player

new_simili = []
for i in tqdm(range(0,len(similarities))):
# for i in range(0,2):
    d = {
        'index':i
        ,'scores':np.sort(normalize(similarities[i,:]))[-11:][::-1][1:]
        ,'reco_player_index': similarities[i,:].argsort()[-11:][::-1][1:]
    }
    new_simili.append(d)
new_simili

/var/folders/tc/0tkkgmdd25n0rh3_pjfhw6mw0000gp/T/ipykernel_23704/570194599.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0,len(similarities))):


  0%|          | 0/19239 [00:00<?, ?it/s]

[{'index': 0,
  'scores': array([98.56, 98.21, 98.01, 97.91, 97.91, 97.88, 97.85, 97.76, 97.63,
         97.57]),
  'reco_player_index': array([ 75,   3,  17,  79,  13, 405, 141,  64,  33,  11])},
 {'index': 1,
  'scores': array([99.14, 98.66, 98.49, 98.39, 98.36, 98.05, 97.92, 97.8 , 97.72,
         97.69]),
  'reco_player_index': array([ 23,   2,  11,  36,  30,   9,  74, 218,  16,  25])},
 {'index': 2,
  'scores': array([98.66, 98.25, 98.06, 97.83, 97.73, 97.58, 97.49, 97.46, 97.34,
         97.34]),
  'reco_player_index': array([   1,   30,   11,   16,  218,   83,    0,    6,  405, 1507])},
 {'index': 3,
  'scores': array([99.06, 98.4 , 98.33, 98.31, 98.19, 98.18, 98.17, 97.92, 97.73,
         97.59]),
  'reco_player_index': array([75, 27, 33, 13, 89, 17,  0, 64, 79, 16])},
 {'index': 4,
  'scores': array([98.84, 98.51, 98.25, 98.2 , 98.01, 97.91, 97.71, 97.66, 97.64,
         97.64]),
  'reco_player_index': array([116,  28,  78, 308,  37,  32, 172, 870, 415, 122])},
 {'index': 5,
 

In [55]:
## get a list of dictionaries : 

list_res = []
for i in range(0,len(res)):
    d = {
        'sofifa_id': int(res.iloc[i]['sofifa_id']),
        'score': res.iloc[i]['score'],
        'index' : i
        }
    list_res.append(d)
list_res

[{'sofifa_id': 231747, 'score': 97.27, 'index': 0},
 {'sofifa_id': 212194, 'score': 97.0, 'index': 1},
 {'sofifa_id': 211110, 'score': 96.95, 'index': 2},
 {'sofifa_id': 220834, 'score': 96.93, 'index': 3},
 {'sofifa_id': 209331, 'score': 96.81, 'index': 4},
 {'sofifa_id': 213345, 'score': 96.75, 'index': 5},
 {'sofifa_id': 202166, 'score': 96.73, 'index': 6},
 {'sofifa_id': 222492, 'score': 96.63, 'index': 7},
 {'sofifa_id': 211300, 'score': 96.6, 'index': 8},
 {'sofifa_id': 165153, 'score': 96.56, 'index': 9}]

In [22]:
sim_array_df =[]
for i in range(0,2):
    d = {
        'player_index':i,
        'scores':np.sort(normalize(similarities[i,:]))[-11:][::-1][1:],
        'reco_player_index': similarities[i,:].argsort()[-11:][::-1][1:]
    }
    sim_array_df.append(d)
sim_array_df

[{'player_index': 0,
  'scores': array([98.56, 98.21, 98.01, 97.91, 97.91, 97.88, 97.85, 97.76, 97.63,
         97.57]),
  'reco_player_index': array([ 75,   3,  17,  79,  13, 405, 141,  64,  33,  11])},
 {'player_index': 1,
  'scores': array([99.14, 98.66, 98.49, 98.39, 98.36, 98.05, 97.92, 97.8 , 97.72,
         97.69]),
  'reco_player_index': array([ 23,   2,  11,  36,  30,   9,  74, 218,  16,  25])}]

In [35]:
res

,score,sofifa_id
index_player,,
6,97.27,231747
344,97.00,212194
41,96.95,211110
182,96.93,220834
17,96.81,209331
64,96.75,213345
429,96.73,202166
132,96.63,222492
342,96.60,211300


In [29]:
sim_df_test = pd.DataFrame.from_dict(new_simili)
sim_df_test.head()

,index,scores,reco_player_index
0,0,"[98.56, 98.21, 98.01, 97.91, 97.91, 97.88, 97....","[75, 3, 17, 79, 13, 405, 141, 64, 33, 11]"
1,1,"[99.14, 98.66, 98.49, 98.39, 98.36, 98.05, 97....","[23, 2, 11, 36, 30, 9, 74, 218, 16, 25]"
2,2,"[98.66, 98.25, 98.06, 97.83, 97.73, 97.58, 97....","[1, 30, 11, 16, 218, 83, 0, 6, 405, 1507]"
3,3,"[99.06, 98.4, 98.33, 98.31, 98.19, 98.18, 98.1...","[75, 27, 33, 13, 89, 17, 0, 64, 79, 16]"
4,4,"[98.84, 98.51, 98.25, 98.2, 98.01, 97.91, 97.7...","[116, 28, 78, 308, 37, 32, 172, 870, 415, 122]"


In [58]:
type(sim_df_test.scores)

pandas.core.series.Series

In [25]:
list_res = []
# index = ...
for i in range(0,10):
    d = {
        'score': sim_df_test.scores[0][i],
        'index' : sim_df_test.reco_player_index[0][i]
        }
    list_res.append(d)
list_res

[{'score': 98.56, 'index': 75},
 {'score': 98.21, 'index': 3},
 {'score': 98.01, 'index': 17},
 {'score': 97.91, 'index': 79},
 {'score': 97.91, 'index': 13},
 {'score': 97.88, 'index': 405},
 {'score': 97.85, 'index': 141},
 {'score': 97.76, 'index': 64},
 {'score': 97.63, 'index': 33},
 {'score': 97.57, 'index': 11}]

In [ ]:
def cosine_recommendation(player,sim_mat,df):
    index = get_index(df,player)
    norm_sim_array= np.sort(normalize(sim_mat[index,:]))[-11:][::-1][1:]
    index_matrix = sim_mat[index,:].argsort()[-11:][::-1][1:]
    dict_simili = {'index_player': index_matrix,'score':norm_sim_array}
    reco_df = pd.DataFrame(dict_simili).set_index('index_player')
    reco_df['sofifa_id'] = df.iloc[reco_df.index]['sofifa_id']
    return reco_df

In [77]:
# simili_import_df = pd.read_csv("../simili_df.csv",index_col=0)
with open("../similarity_matrix.pickle", 'rb') as file:
    sim_matrix_df = pickle.load(file)

In [78]:
sim_matrix_df.head()

,index,scores,reco_player_index
0,0,"[98.56, 98.21, 98.01, 97.91, 97.91, 97.88, 97....","[75, 3, 17, 79, 13, 405, 141, 64, 33, 11]"
1,1,"[99.14, 98.66, 98.49, 98.39, 98.36, 98.05, 97....","[23, 2, 11, 36, 30, 9, 74, 218, 16, 25]"
2,2,"[98.66, 98.25, 98.06, 97.83, 97.73, 97.58, 97....","[1, 30, 11, 16, 218, 83, 0, 6, 405, 1507]"
3,3,"[99.06, 98.4, 98.33, 98.31, 98.19, 98.18, 98.1...","[75, 27, 33, 13, 89, 17, 0, 64, 79, 16]"
4,4,"[98.84, 98.51, 98.25, 98.2, 98.01, 97.91, 97.7...","[116, 28, 78, 308, 37, 32, 172, 870, 415, 122]"


In [57]:
type(simili_import_df.scores)

pandas.core.series.Series

In [79]:
# for i in range(0,3):
print(sim_matrix_df.scores[1])
type(sim_matrix_df.scores[1])

[99.14 98.66 98.49 98.39 98.36 98.05 97.92 97.8  97.72 97.69]


numpy.ndarray

In [82]:
index_search = 0 

list_res=[]
for i in range(0,10):
    d = {
        'index_search' : index_search,
        'index' : sim_matrix_df.reco_player_index[0][i],
        'score': sim_matrix_df.scores[0][i]
        }
    list_res.append(d)
list_res

# dict_simili = {'index_player': index_matrix,'score':norm_sim_array}

[{'index_search': 0, 'index': 75, 'score': 98.56},
 {'index_search': 0, 'index': 3, 'score': 98.21},
 {'index_search': 0, 'index': 17, 'score': 98.01},
 {'index_search': 0, 'index': 79, 'score': 97.91},
 {'index_search': 0, 'index': 13, 'score': 97.91},
 {'index_search': 0, 'index': 405, 'score': 97.88},
 {'index_search': 0, 'index': 141, 'score': 97.85},
 {'index_search': 0, 'index': 64, 'score': 97.76},
 {'index_search': 0, 'index': 33, 'score': 97.63},
 {'index_search': 0, 'index': 11, 'score': 97.57}]

In [83]:
my_reco = pd.DataFrame(list_res)

In [86]:

my_reco['sofifa_id']=df22.iloc[my_reco.index]['sofifa_id']
my_reco

,index_search,index,score,sofifa_id
0,0,75,98.56,158023
1,0,3,98.21,188545
2,0,17,98.01,20801
3,0,79,97.91,190871
4,0,13,97.91,192985
5,0,405,97.88,200389
6,0,141,97.85,231747
7,0,64,97.76,167495
8,0,33,97.63,192448
9,0,11,97.57,202126


In [88]:
my_reco.to_dict()

{'index_search': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0},
 'index': {0: 75,
  1: 3,
  2: 17,
  3: 79,
  4: 13,
  5: 405,
  6: 141,
  7: 64,
  8: 33,
  9: 11},
 'score': {0: 98.56,
  1: 98.21,
  2: 98.01,
  3: 97.91,
  4: 97.91,
  5: 97.88,
  6: 97.85,
  7: 97.76,
  8: 97.63,
  9: 97.57},
 'sofifa_id': {0: 158023,
  1: 188545,
  2: 20801,
  3: 190871,
  4: 192985,
  5: 200389,
  6: 231747,
  7: 167495,
  8: 192448,
  9: 202126}}

In [87]:
list_res = []
for i in range(0,len(my_reco)):
    d = {
        'sofifa_id': int(my_reco.iloc[i]['sofifa_id']),
        'score': my_reco.iloc[i]['score'],
        'index' : i
        }
    list_res.append(d)
list_res

[{'sofifa_id': 158023, 'score': 98.56, 'index': 0},
 {'sofifa_id': 188545, 'score': 98.21, 'index': 1},
 {'sofifa_id': 20801, 'score': 98.01, 'index': 2},
 {'sofifa_id': 190871, 'score': 97.91, 'index': 3},
 {'sofifa_id': 192985, 'score': 97.91, 'index': 4},
 {'sofifa_id': 200389, 'score': 97.88, 'index': 5},
 {'sofifa_id': 231747, 'score': 97.85, 'index': 6},
 {'sofifa_id': 167495, 'score': 97.76, 'index': 7},
 {'sofifa_id': 192448, 'score': 97.63, 'index': 8},
 {'sofifa_id': 202126, 'score': 97.57, 'index': 9}]

In [30]:
# np.savetxt("../simili_test.csv", similarities, delimiter=",")
! pwd

/Users/user/code/riadh-anis/opti_recruit/notebooks


In [32]:
sim_df_test.to_csv('../simili_df.csv')

In [76]:
with open(r'../similarity_matrix.pickle', 'wb') as file:
        pickle.dump(sim_df_test, file)

In [ ]:
for i,v in cosine_recommendation('K. Havertz').items():
    print(df22_new.iloc[i]['short_name'], v)

In [1]:
import requests

opti_api_url = 'http://127.0.0.1:8000/similarities?player_id=158023'

response = requests.get(
    opti_api_url
).json()

print(response)

[{'sofifa_id': 158023, 'score': 98.56, 'index': 0}, {'sofifa_id': 188545, 'score': 98.21, 'index': 1}, {'sofifa_id': 20801, 'score': 98.01, 'index': 2}, {'sofifa_id': 190871, 'score': 97.91, 'index': 3}, {'sofifa_id': 192985, 'score': 97.91, 'index': 4}, {'sofifa_id': 200389, 'score': 97.88, 'index': 5}, {'sofifa_id': 231747, 'score': 97.85, 'index': 6}, {'sofifa_id': 167495, 'score': 97.76, 'index': 7}, {'sofifa_id': 192448, 'score': 97.63, 'index': 8}, {'sofifa_id': 202126, 'score': 97.57, 'index': 9}]


### KNN Similarity

In [ ]:
from sklearn.neighbors import NearestNeighbors

reco = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(test_df)



In [ ]:
reco.kneighbors(test_df)

In [ ]:
players_recommended_index = reco.kneighbors(test_df)[1]

In [ ]:
players_recommended_scores = reco.kneighbors(test_df)[0]

In [ ]:
players_recommended_scores[index][1:]


In [ ]:
print(f'Here are 10 players similar to K.Mbappe :' '\n')
for i in players_recommended[index][1:]:
    print(df22_new.iloc[i]['short_name'])